In [47]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing copkmeans.egg-info\PKG-INFO


C:\Users\User\anacondatake2\Lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
C:\Users\User\anacondatake2\Lib\site-packages\setuptools\_distutils\cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://g

## Imports and reading data

In [48]:
import pandas as pd
import numpy as np #for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')#for model-building
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer#for word embedding
# Word2Vec (may deal with this later)
#! pip install gensim
import gensim
from gensim.models import Word2Vec
from copkmeans.cop_kmeans import cop_kmeans

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
df = pd.read_csv('nrao_projects_unlabeled.csv')
#df['target'] = pd.get_dummies(df['fs_type'], dtype=int)['line']
df['text'] = df.project_title + df.project_abstract
df['text'] = df['text'].astype(str)
df.head()

,project_title,project_abstract,text
0,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,Fifty AU STudy of the chemistry in the disk/en...
1,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,COMPASS: Complex Organic Molecules in Protosta...
2,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,ALCHEMI: the ALMA Comprehensive High-resolutio...
3,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,ALMA JELLY - Survey of Nearby Jellyfish and Ra...
4,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,Bulge symmetry or not? The hidden dynamics of ...


In [50]:
df_train = df

## Text Preprocessing

In [51]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [52]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df_train['clean_text'] = df_train['text'].apply(lambda x: finalpreprocess(x))
df_train.head()

,project_title,project_abstract,text,clean_text
0,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,Fifty AU STudy of the chemistry in the disk/en...,fifty au study chemistry disk envelope system ...
1,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,COMPASS: Complex Organic Molecules in Protosta...,compass complex organic molecule protostars al...
2,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,ALCHEMI: the ALMA Comprehensive High-resolutio...,alchemi alma comprehensive high resolution ext...
3,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,alma jelly survey nearby jellyfish ram pressur...
4,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,Bulge symmetry or not? The hidden dynamics of ...,bulge symmetry hidden dynamic far sidea radio ...


## Extracting Vectors

In [17]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
#X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"],df_train["target"],test_size=0.2,shuffle=True)#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in df_train["clean_text"]]  
#X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [18]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(df_train) 
#X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
df['clean_text_tok']=[nltk.word_tokenize(i) for i in df['clean_text']]

model = Word2Vec(df['clean_text_tok'],min_count=1) 
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors)) ## CHANGE model.wv._____

modelw = MeanEmbeddingVectorizer(w2v)

# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
#X_test_vectors_w2v = modelw.transform(X_test_tok) # CHANGE var name

In [53]:
X_train_vectors_w2v

array([[-0.43659714, -0.06955937,  0.01061684, ..., -0.24160674,
         0.10435159, -0.14715746],
       [-0.22946283, -0.0376877 ,  0.29437652, ..., -0.30333504,
         0.32472518,  0.03589132],
       [ 0.21404347,  0.25308576,  0.3628998 , ..., -0.23857297,
         0.21672946,  0.04336225],
       ...,
       [-0.16680756,  0.27049297,  0.17251848, ...,  0.0441983 ,
         0.08360934, -0.15190113],
       [-0.13058499,  0.36921644,  0.07829084, ...,  0.11240532,
        -0.00583314, -0.13323224],
       [ 0.04436465,  0.2864441 ,  0.15862204, ..., -0.20103566,
         0.09721981, -0.02091009]], dtype=float32)

In [45]:
input_matrix = X_train_vectors_tfidf
must_link = [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), 
             (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), 
             (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), 
             (0, 22), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), 
             (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), 
             (0, 38), (0, 39), (0, 40), (0, 41), (0, 42), (0, 43), (0, 44), 
             (0, 45), (0, 46), (0, 47), (0, 48), (0, 49), (0, 50), (0, 51), 
             (0, 52), (0, 53), (0, 54), (0, 55), (0, 56), (0, 57), (0, 58), 
             (0, 59), (0, 60), (0, 61), (0, 62), (0, 63), (0, 64), (0, 65), 
             (0, 66), (0, 67), (0, 68), (0, 69), (0, 70), (0, 71), (0, 72), 
             (0, 73), (0, 74), (0, 75), (0, 76), (0, 77), (0, 78), (0, 79), 
             (0, 80), (0, 81), (0, 82), (0, 83), (0, 84), (0, 85), (0, 86), (0, 87),
             (0, 88), (0, 89), (0, 90)]
cannot_link = [(0, 23), (0, 757), (0, 1123), (0, 1124), (0, 1125), (0, 1126),
              (0, 1127), (0, 1128), (0, 1129), (0, 1130), (0, 1131), (0, 1132),
              (0, 1133), (0, 1134), (0, 1135), (0, 1136), (0, 1137), (0, 1138),
              (0, 1139), (0, 1140), (0, 1141), (0, 1142), (0, 1143), (0, 1144),
              (0, 1145), (0, 1146), (0, 1147),(0, 1148), (0, 1149), (0, 1150),
              (0, 1151), (0, 1152), (0, 1153), (0, 1154), (0, 1155), (0, 1156),
              (0, 1157), (0, 1158), (0, 1159), (0, 1160), (0, 1161), (0, 1162),
              (0, 1163), (0, 1164), (0, 1165), (0, 1166), (0, 1167), (0, 1168),
              (0, 1169), (0, 1170), (0, 1171), (0, 1172), (0, 1173), (0, 1174),
              (0, 1175), (0, 1176), (0, 1177),(0, 1178), (0, 1179), (0, 1180),
              (0, 1181), (0, 1182), (0, 1183),(0, 1184), (0, 1185), (0, 1186),
              (0, 1187), (0, 1188), (0, 1189), (0, 1190), (0, 1191), (0, 1192),
              (0, 1193), (0, 1194), (0, 1195), (0, 1196), (0, 1197), (0, 1198),
              (0, 1199), (0, 1200), (0, 1201), (0, 1202), (0, 1203), (0, 1204),
              (0, 1205), (0, 1206), (0, 1207), (0, 1208), (0, 1209), (0, 1210)]

In [46]:
clusters, centers = cop_kmeans(dataset=input_matrix, k=2, ml=must_link,cl=cannot_link)

TypeError: sparse array length is ambiguous; use getnnz() or shape[0]

In [38]:
true_df = pd.read_csv('nrao_projects.csv')
for i in range(len(true_df)):
    if true_df.iloc[i,3] == "line":
        true_df.iloc[i,3] = 1
    else:
        true_df.iloc[i,3] = 0

In [41]:
sum(abs(true_df.iloc[:,3] - clusters))

2048

In [44]:
1-(2048/len(true_df))

0.5477031802120141